<a href="https://colab.research.google.com/github/friedelj/ComputerVision_Team2/blob/main/Team2_Computer_Vision_draft2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [15]:
import os
import random
import torch
import cv2
from torchvision.transforms import ToTensor
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog, Label, Button
import numpy as np
from pycocotools.coco import COCO
from torchvision.models.detection import maskrcnn_resnet50_fpn
import requests
from tqdm import tqdm  # For progress bar
import requests
import tarfile
import zipfile
import gradio as gr

In [16]:
# Set the base directory where the dataset will be downloaded
base_dir = "C:\\Users\\josep\\COCO"  # Replace with your desired path

# URLs for the COCO 2017 dataset
urls = {
    "train2017": "http://images.cocodataset.org/zips/train2017.zip",
    "val2017": "http://images.cocodataset.org/zips/val2017.zip",
    "annotations": "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
}

In [17]:
# Create the directory if it doesn't exist
os.makedirs(base_dir, exist_ok=True)

In [18]:
def download_and_extract(url, dest_folder):
    filename = os.path.join(dest_folder, url.split("/")[-1])
    # Download file
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        with open(filename, 'wb') as file, tqdm(
            desc=filename,
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(chunk_size=1024):
                bar.update(len(data))
                file.write(data)

    # Extract file
    print(f"Extracting {filename}...")
    import zipfile
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(dest_folder)

In [19]:
def download_and_extract(url, base_dir):
    """
    Downloads and extracts a file from a given URL.
    Supports .zip and .tar.gz formats.

    Args:
        url (str): The URL of the file to download.
        base_dir (str): Directory to extract the files into.
    """
    # Get the filename from the URL
    filename = os.path.basename(url)
    filepath = os.path.join(base_dir, filename)

    # Download the file
    if not os.path.exists(filepath):
        print(f"Downloading {filename}...")
        response = requests.get(url, stream=True)
        with open(filepath, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)

    # Extract the file
    print(f"Extracting {filename}...")
    if filename.endswith(".zip"):
        with zipfile.ZipFile(filepath, "r") as zip_ref:
            zip_ref.extractall(base_dir)
    elif filename.endswith((".tar.gz", ".tgz")):
        with tarfile.open(filepath, "r:gz") as tar_ref:
            tar_ref.extractall(base_dir)
    else:
        print(f"Unsupported file type for {filename}")

In [20]:
for name, url in urls.items():
    download_and_extract(url, base_dir)

Extracting train2017.zip...
Extracting val2017.zip...
Extracting annotations_trainval2017.zip...


In [21]:
# Load YOLOv5 model
def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s')

In [22]:
# Load Mask R-CNN model
def load_mask_rcnn_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

In [23]:
# COCO dataset initialization
def initialize_coco_dataset(dataset_path, annotation_file):
    coco = COCO(annotation_file)
    image_ids = coco.getImgIds()
    images = coco.loadImgs(image_ids)
    return coco, images

In [24]:
# People counting logic using bounding box overlap
def count_people(bboxes):
    count = 0
    for i, bbox1 in enumerate(bboxes):
        overlap = False
        for j, bbox2 in enumerate(bboxes):
            if i != j and calculate_iou(bbox1, bbox2) > 0.5:
                overlap = True
                break
        if not overlap:
            count += 1
    return count

In [25]:
# IOU calculation
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)

    inter_area = max(0, xb - xa) * max(0, yb - ya)
    box1_area = w1 * h1
    box2_area = w2 * h2

    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

In [26]:
# Randomly select images
def select_random_images(images, num_images=6):
    return random.sample(images, num_images)

In [27]:
# GUI for displaying images and counts
class PeopleCountingApp:
    def __init__(self, root, coco, images, yolov5_model, mask_rcnn_model, dataset_path):
        self.root = root
        self.coco = coco
        self.images = images
        self.yolov5_model = yolov5_model
        self.mask_rcnn_model = mask_rcnn_model
        self.dataset_path = dataset_path
        self.image_labels = []

        self.setup_ui()

    def setup_ui(self):
        self.label = Label(self.root, text="People Counting System")
        self.label.pack()

        self.select_button = Button(self.root, text="Select Random Images", command=self.show_random_images)
        self.select_button.pack()

    def show_random_images(self):
        # Clear existing labels
        for label in self.image_labels:
            label.destroy()
        self.image_labels = []

        # Select random images
        selected_images = select_random_images(self.images)

        for img_info in selected_images:
            img_path = os.path.join(self.dataset_path, img_info['file_name'])
            image = Image.open(img_path)
            count = self.process_image(img_path)

            # Resize for display
            image = image.resize((300, 300))
            photo = ImageTk.PhotoImage(image)

            img_label = Label(self.root, image=photo)
            img_label.photo = photo  # Keep reference
            img_label.pack()

            count_label = Label(self.root, text=f"People Count: {count}")
            count_label.pack()

            self.image_labels.append(img_label)
            self.image_labels.append(count_label)

    def process_image(self, img_path):
        image = cv2.imread(img_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_tensor = ToTensor()(image_rgb).unsqueeze(0)

        # YOLOv5 detection
        yolov5_results = self.yolov5_model(image)
        yolo_bboxes = [x[:4] for x in yolov5_results.xyxy[0].cpu().numpy() if int(x[-1]) == 0]  # Class 0: Person

        # Mask R-CNN detection
        mask_rcnn_results = self.mask_rcnn_model(image_tensor)[0]
        mask_rcnn_bboxes = mask_rcnn_results['boxes'][mask_rcnn_results['labels'] == 1].detach().cpu().numpy()

        # Combine results and count
        combined_bboxes = list(yolo_bboxes) + list(mask_rcnn_bboxes)
        return count_people(combined_bboxes)

In [28]:
# Load models and dataset
def initialize_coco_dataset(dataset_path, annotation_file):
    from pycocotools.coco import COCO
    coco = COCO(annotation_file)
    images = coco.getImgIds()
    return coco, images

def load_yolov5_model():
    import torch
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', trust_repo=True)
    return model

In [29]:
def load_mask_rcnn_model():
    import torchvision
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

def count_people(image):
    # Perform inference using YOLOv5 and Mask R-CNN models
    results = yolov5_model(image)
    detections = results.pandas().xyxy[0]  # Get detections in pandas format
    people_count = len(detections[detections['name'] == 'person'])

    return f"Number of people detected: {people_count}"

In [35]:
# Setup paths
dataset_path = '/content/'
annotation_file = '/content/instances_val2017.json'

In [38]:
# Load COCO dataset and models
coco, images = initialize_coco_dataset(dataset_path, annotation_file)
yolov5_model = load_yolov5_model()
mask_rcnn_model = load_mask_rcnn_model()

loading annotations into memory...
Done (t=1.28s)
creating index...
index created!


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2024-11-30 Python-3.10.12 torch-2.5.1+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 323MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:01<00:00, 129MB/s]


In [39]:
# Gradio Interface
iface = gr.Interface(fn=count_people, inputs="image", outputs="text", title="People Counting App")
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9727e7bcd0938dd9f3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
